In [26]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.stats import entropy

df = pd.read_csv("../raw/sampled_hotel_1000.csv")
df


,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02


# length

In [29]:
def count_words(column):
    return column.apply(lambda x: len(str(x).split()))

# 각 열의 단어 수 계산하여 새로운 열 추가
df['title_length'] = count_words(df['review_title'])
df['text_length'] = count_words(df['Review_Text'])

# 결과 출력
df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,hotel_grade,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,title_length,text_length
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,3,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02,9,31
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,3,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02,3,47
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,3,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16,24,7
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,6,34
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,4,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02,8,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",2,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02,7,100
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,0,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02,18,7
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,3,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,4,3
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,4,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02,7,49


# time_lapesd

In [32]:
# Posted_Date와 Crawled_date를 datetime 형식으로 변환
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], format='%Y-%m-%d', errors='coerce')
df['Crawled_date'] = pd.to_datetime(df['Crawled_date'], format='%Y-%m-%d', errors='coerce')

# 날짜 차이 계산 (Crawled_date - Posted_Date)
df['time_lapsed'] = (df['Crawled_date'] - df['Posted_Date']).dt.days  # 일(day) 단위로 계산

df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,employee_friendliness_score,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,title_length,text_length,time_lapsed
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,8.7,7.6,8.2,8.0,7.5,9.1,2024-12-02,9,31,173
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,8.1,6.9,7.3,7.3,7.0,9.4,2024-12-02,3,47,5
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,9.0,7.7,8.2,8.2,8.0,9.4,2024-12-16,24,7,200
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,6,34,64
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.7,8.3,8.7,8.8,8.1,8.9,2024-12-02,8,13,388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,8.6,7.6,8.4,8.1,7.9,9.4,2024-12-02,7,100,98
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,8.1,7.2,7.9,7.9,7.5,9.6,2024-12-02,18,7,1017
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,9.1,8.4,8.7,8.7,8.2,9.7,2024-12-02,4,3,881
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,8.7,8.0,8.4,8.5,7.4,9.1,2024-12-02,7,49,523


# star_dev

In [35]:
df['Deviation of star ratings'] = abs(df['Rating'] - df['Average_Rating'])
df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,facility_score,cleanliness_score,comfort_score,value_for_money_score,location_score,Crawled_date,title_length,text_length,time_lapsed,Deviation of star ratings
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,7.6,8.2,8.0,7.5,9.1,2024-12-02,9,31,173,1.3
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,6.9,7.3,7.3,7.0,9.4,2024-12-02,3,47,5,3.0
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,7.7,8.2,8.2,8.0,9.4,2024-12-16,24,7,200,1.8
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,8.4,8.7,8.7,8.2,9.7,2024-12-02,6,34,64,0.4
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.3,8.7,8.8,8.1,8.9,2024-12-02,8,13,388,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,7.6,8.4,8.1,7.9,9.4,2024-12-02,7,100,98,0.8
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,7.2,7.9,7.9,7.5,9.6,2024-12-02,18,7,1017,0.4
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,8.4,8.7,8.7,8.2,9.7,2024-12-02,4,3,881,0.6
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,8.0,8.4,8.5,7.4,9.1,2024-12-02,7,49,523,0.1


# readability

In [38]:
import textstat


# 4. Readability 점수를 계산할 열 이름 설정
text_column = "Review_Text"  # CSV 파일의 텍스트가 저장된 열 이름

# 5. 새로운 열 추가 및 점수 계산
df["FOG Index"] = df[text_column].apply(lambda text: textstat.gunning_fog(str(text)))
df["Flesch Reading Ease"] = df[text_column].apply(lambda text: textstat.flesch_reading_ease(str(text)))

df

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,comfort_score,value_for_money_score,location_score,Crawled_date,title_length,text_length,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,8.0,7.5,9.1,2024-12-02,9,31,173,1.3,13.69,65.39
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,7.3,7.0,9.4,2024-12-02,3,47,5,3.0,22.20,40.69
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,8.2,8.0,9.4,2024-12-16,24,7,200,1.8,2.80,89.75
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,8.7,8.2,9.7,2024-12-02,6,34,64,0.4,18.31,53.89
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.8,8.1,8.9,2024-12-02,8,13,388,1.3,5.20,100.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,8.1,7.9,9.4,2024-12-02,7,100,98,0.8,41.60,3.82
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,7.9,7.5,9.6,2024-12-02,18,7,1017,0.4,8.51,64.37
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,8.7,8.2,9.7,2024-12-02,4,3,881,0.6,14.53,34.59
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,8.5,7.4,9.1,2024-12-02,7,49,523,0.1,20.42,55.58


# Depth

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,value_for_money_score,location_score,Crawled_date,title_length,text_length,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease,ContentDepth
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,7.5,9.1,2024-12-02,9,31,173,1.3,13.69,65.39,0.517676
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,7.0,9.4,2024-12-02,3,47,5,3.0,22.20,40.69,1.365924
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,8.0,9.4,2024-12-16,24,7,200,1.8,2.80,89.75,1.250888
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,8.2,9.7,2024-12-02,6,34,64,0.4,18.31,53.89,0.466643
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.1,8.9,2024-12-02,8,13,388,1.3,5.20,100.58,1.527434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,7.9,9.4,2024-12-02,7,100,98,0.8,41.60,3.82,2.247030
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,7.5,9.6,2024-12-02,18,7,1017,0.4,8.51,64.37,1.250676
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,8.2,9.7,2024-12-02,4,3,881,0.6,14.53,34.59,1.832371
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,7.4,9.1,2024-12-02,7,49,523,0.1,20.42,55.58,1.364491


# Breadth

,Unnamed: 0,Hotel_Name,Review_Text,Posted_Date,Rating,Average_Rating,Num_of_Ratings,Helpfulness,is_photo,review_title,...,location_score,Crawled_date,title_length,text_length,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease,ContentDepth,ContentBreadth
0,0,tavistock,We have stayed here often and it is ideal Loca...,2024-06-12,9.0,7.7,5932,0,0,Will continue to stay here whenever we visit L...,...,9.1,2024-12-02,9,31,173,1.3,13.69,65.39,0.517676,1.935204
1,1,stgileshotel,"Its in a very convenient location, my mom whos...",2024-11-27,10.0,7.0,14989,0,0,Cozy and comfortable,...,9.4,2024-12-02,3,47,5,3.0,22.20,40.69,1.365924,1.046359
2,2,montanahotel,The staff were friendlyThe very small bathroom,2024-05-30,6.0,7.8,6248,0,0,We were on the 3rd floor were not told prior ...,...,9.4,2024-12-16,24,7,200,1.8,2.80,89.75,1.250888,1.292763
3,3,z-trafalgar,"Good location, was a 10 minute walk from the t...",2024-09-29,8.0,8.4,13923,0,0,Good functional hotel in great location,...,9.7,2024-12-02,6,34,64,0.4,18.31,53.89,0.466643,0.920339
4,4,blakemore,You hear a lot of noise from the other room do...,2023-11-10,7.0,8.3,12340,0,1,Great stay in a clean and tidy hotel,...,8.9,2024-12-02,8,13,388,1.3,5.20,100.58,1.527434,1.582295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,crestfieldhotel,"The room was definitely clean, which is the mo...",2024-08-26,7.0,7.8,6515,0,0,"Noisy and small, but clean and friendly",...,9.4,2024-12-02,7,100,98,0.8,41.60,3.82,2.247030,0.532484
996,996,zedwell-trocaderor,modern look with nice finishroom very basic,2022-02-19,7.0,7.4,39497,0,0,great location which is what supports the pric...,...,9.6,2024-12-02,18,7,1017,0.4,8.51,64.37,1.250676,0.583498
997,997,z-trafalgar,location was perfect,2022-07-05,9.0,8.4,13923,0,0,great stay and location,...,9.7,2024-12-02,4,3,881,0.6,14.53,34.59,1.832371,1.250667
998,998,radissonblugrafton,Big room with a desk and a table Good breakfas...,2023-06-28,8.0,7.9,9315,0,0,Nice rooms but very noisy over night,...,9.1,2024-12-02,7,49,523,0.1,20.42,55.58,1.364491,1.401397


# 파일 저장

In [45]:
df.to_csv("../temp/sampled_hotel_1000_calculated.csv")